In [1]:
import pandas as pd
import numpy as np

import pandas_gbq
import pydata_google_auth
import logging

SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform'
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    auth_local_webserver=True,
)

logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

def execute(sql):
    res = pandas_gbq.read_gbq(
        sql,
        project_id='playgendary-bi',
        credentials=credentials,
    )
    return res

# AB-тесты: общее

<i>При дизайне АБ-теста, стоит рекомендовать продюсеру дизайнить тест с двумя группами: контрольной и тестовой, т.к. поправка на множественную проверку гипотез сильно увеличивает необходимое кол-во человек в группу.</i><br>
https://en.wikipedia.org/wiki/Multiple_comparisons_problem

## 1. Расчет параметров АБ-теста

In [2]:
import scipy as sp
from scipy.stats import norm
from itertools import product

def n(mean, mean_ch, std, a, b=0.8, n1=1, n2=1):
    '''
    mean – среднее
    mean_ch – доля изменения среднего, относительная разница между метриками
    std – стандартное отклонение
    a – уровень значимости, т.е. вероятность ошибки первого рода
    b – мощность, т.е. 1 - вероятность ошибки второго рода, по дефолту закладываем 20% => b=0.8
    n1:n2 – пропорция между группами 
    '''
    n = (n1 + n2) / (n1 * n2) * ( ((norm.ppf(1 - a/2) + norm.ppf(b))) * std / (mean_ch * mean) )**2
    
    return int(n)

def num(mean, std, mean_ch, a, b=0.8, n1=1, n2=1): 
    num_results = [
        {
            'mean change': m,
            'a': a,   
            'n': n(mean, m, std, a, b=b, n1=n1, n2=n2)
        }
        for m, a in product(mean_ch, a)
    ]
    return pd.DataFrame(num_results).pivot('a', 'mean change', 'n')

In [3]:
mean_ch = [.05, .1, .15]
a = [.05, .1, .2]

### Для примера выгрузим данные из FireBase "почасового" ретеншена для HD Android

In [4]:
project = 'home-design-749c0.analytics_207534348.events_*'
start_install_date = '2019-11-04'
end_install_date =  '2019-12-01'

In [5]:
sql_retention = f'''
select
    user_id
    , day
    , ifnull(retention_hours, 0) retention_hours
from
(
      select distinct
          day
          , geo.country country
          , user_pseudo_id user_id
          , platform
      from
          `{project}`
      cross join
          unnest(generate_array(1, 7)) as day 
      where    
          _table_suffix between replace('{start_install_date}', '-', '') and
                                replace('{end_install_date}', '-', '')
          and event_name = 'first_open'
) i
left join
(
  select distinct
      cast(floor((event_timestamp - user_first_touch_timestamp) / (1000000 * 86400)) as int64) day
      , user_pseudo_id user_id
      , 1 retention_hours
  from
      `{project}`
  where    
      _table_suffix between replace('{start_install_date}', '-', '') and
                                    replace(cast(date_add('{end_install_date}', interval 4 day) as string),
                                            '-', '')
      and event_name = 'user_engagement'
      and event_timestamp >= user_first_touch_timestamp
)
using (user_id, day)
'''
df_retention = execute(sql_retention)

Requesting query... 
Query running...
Job ID: b7675470-7711-4308-bdf3-03d6d7433050
Query done.
Processed: 119.4 MB Billed: 120.0 MB
Standard price: $0.00 USD

Got 30877 rows.



In [6]:
df_retention.sample(3)

,user_id,day,retention_hours
20753,91b661925e9fcce6029063f4745808e0,5,0
2985,25a541794aac0b016dc3f1e8a2d49b4a,1,0
24178,ccc196be23742c3368802c3cc52b9e40,6,0


### Расчитаем параметры АБ-теста

In [7]:
(df_retention[df_retention['day'] == 1].retention_hours.mean(),
 df_retention[df_retention['day'] == 7].retention_hours.mean())

(0.2189979596463387, 0.07685332124234867)

In [9]:
## для R1
num(df_retention[df_retention['day'] == 1].retention_hours.mean(), 
    df_retention[df_retention['day'] == 1].retention_hours.std(), mean_ch, a)

mean change,0.05,0.10,0.15
a,,,
0.05,22397,5599,2488
0.10,17642,4410,1960
0.20,12863,3215,1429


In [10]:
## для R7
num(df_retention[df_retention['day'] == 7].retention_hours.mean(), 
    df_retention[df_retention['day'] == 7].retention_hours.std(), mean_ch, a)

mean change,0.05,0.10,0.15
a,,,
0.05,75440,18860,8382
0.10,59424,14856,6602
0.20,43327,10831,4814


Например:
- при a = 0.05 и детектируемой относительной разнице в 5% для метрики R7 необходимо 75440 человек в группу
- при a = 0.2 и детектируемой относительной разнице в 15% для метрики R1 необходимо 1429 человек в группу

<i>Всегда стоит проверять полученные цифры на адекватность, сравнивать с расчетами предыдыдущих тестов. Также можно использовать след. калькуляторы для проверки своих результатов:</i>

- https://docs.google.com/spreadsheets/d/1n6D14bVEfNcpH8nFfQNaFGp251p3lQa96loXqF2Uf3Q
- https://cxl.com/ab-test-calculator/

<i>Если получилось оч. большое кол-во пользователей в группу и ошибки нет, то стоит повторно с продюсером обсудить параметры, возможно избранная метрика слишком требовательна.</i>

## 2. Анализ АБ-теста

### Для примера выгрузим данные по тесту  Hit the Light Android для рекламного RPI7 и ретеншена
*события принадлежности к АБ-группе пользователя может быть реализовано по-разному, в данном случае реализация была через AF*

In [11]:
project_ab = 'com.happymagenta.hitlight'
start_install_date_ab = '2019-09-27'
end_install_date_ab =  '2019-10-10'

In [12]:
sql_ab = f'''
select
    appsflyer_id
    , ab_group
    , ifnull(rev_ad, 0) rev_ad
    , ifnull(r7, 0) r7
from
(
    select
        appsflyer_id
        , event_value ab_group
    from
      `playgendary-bi.appsflyer_datalocker_2.inapps_*`
    where
      _table_suffix between replace('{start_install_date_ab}', '-', '') and
                                    replace('{end_install_date_ab}', '-', '')
      and app_id = '{project_ab}'
      and event_name = 'AB_group'
)
left join
(
    select
        appsflyerId appsflyer_id
        , sum(revenue) rev_ad
    from
        `playgendary-bi.aggregated_data.daily_ad_revenue` 
    where 
        installDate between '{start_install_date_ab}' and '{end_install_date_ab}'
        and appId = '{project_ab}'
        and date_diff(revenueDate, installDate, day) <= 7
    group by
        appsflyer_id
)
using(appsflyer_id)
left join
(
    select distinct
        appsflyer_id
        , 1 r7
    from
        `playgendary-bi.appsflyer_datalocker_2.sessions_*`
    where 
        _table_suffix between replace('{start_install_date_ab}', '-', '') and
                              replace(cast(date_add('{end_install_date_ab}', interval 8 day) as string), '-', '')
        and app_id = '{project_ab}'
        and date_diff(date(parse_datetime('%Y-%m-%d %H:%M:%S', event_time)),
                      date(parse_datetime('%Y-%m-%d %H:%M:%S', install_time)), day) = 7
 )
using(appsflyer_id)
'''

In [13]:
df_ab = execute(sql_ab)

Requesting query... 
Query running...
Job ID: 8769bf8d-7797-408f-af04-0a5757504b68
  Elapsed 7.56 s. Waiting...
  Elapsed 9.92 s. Waiting...
  Elapsed 12.08 s. Waiting...
Query done.
Processed: 289.5 GB Billed: 289.5 GB
Standard price: $1.41 USD

Got 326320 rows.

Total time taken 32.84 s.
Finished at 2019-12-11 12:23:01.


In [14]:
df_ab.sample(3)

,appsflyer_id,ab_group,rev_ad,r7
223038,1570288555196-9150863444262360672,"{""af_param_1"":""control_sub""}",0.18779,1
138859,1570255640914-1685062855802389442,"{""af_param_1"":""test_sub""}",0.00397,0
280647,1570444610266-2991043334730753179,"{""af_param_1"":""control_sub""}",0.00005,0


In [15]:
#df_ab.to_csv('ab.csv', index=False)

In [16]:
df_ab.groupby('ab_group').appsflyer_id.count()

ab_group
{"af_param_1":"control_sub"}    162639
{"af_param_1":"test_sub"}       163681
Name: appsflyer_id, dtype: int64

*Убеждаемся, что в тесте нет пересечений пользователей между группами, убираем дубликаты событий<br>
Иногда, в целом по событиям, стоит дополнительно проверить правильно ли работает тест, приходят ли тестовой группе события, которые должны приходить и т.д.*

In [17]:
df_ab.drop_duplicates(subset=['appsflyer_id', 'ab_group'], inplace=True)

## Оценка метрик и различий между группами

In [18]:
df_ab.groupby('ab_group')['rev_ad', 'r7'].mean()

,rev_ad,r7
ab_group,,
"{""af_param_1"":""control_sub""}",0.041503,0.026621
"{""af_param_1"":""test_sub""}",0.042715,0.027290


In [19]:
#относительная разница между ad RPI7 тестовой и контрольной группами
1 - df_ab.groupby('ab_group')['rev_ad'].mean()[1] / df_ab.groupby('ab_group')['rev_ad'].mean()[0]

-0.029217202023584576

In [20]:
#относительная разница между R7 тестовой и контрольной группами
1 - df_ab.groupby('ab_group')['r7'].mean()[1] / df_ab.groupby('ab_group')['r7'].mean()[0]

-0.025135095442821553

In [21]:
sp.stats.ttest_ind(df_ab[df_ab['ab_group'].str.contains('test')]['rev_ad'],
                   df_ab[df_ab['ab_group'].str.contains('control')]['rev_ad'], equal_var=False)

Ttest_indResult(statistic=2.061604341763111, pvalue=0.03924621786450064)

### Сравниваем полученное p_value с тем, что было заложено при расчете параметров АБ-теста, в данном тесте 0.05

*p_value = 0.04 => мы можем отклонить нулевую гипотезу о равенстве средних контрольной и тестовой групп. т.е. делаем вывод о том, что разница между группами статистически значима*

In [22]:
sp.stats.ttest_ind(df_ab[df_ab['ab_group'].str.contains('test')]['r7'],
                   df_ab[df_ab['ab_group'].str.contains('control')]['r7'], equal_var=False)

Ttest_indResult(statistic=1.1799814883945832, pvalue=0.2380084378875563)

*p_value = 0.24 => мы не можем отклонить нулевую гипотезу о равенстве средних контрольной и тестовой групп. т.е. делаем вывод о том, что разница между группами статистически незначима*

## Вывод по АБ-тесту

- Retention D7 в тестовой группе **не значимо** выше, чем в контрольной на 2,5%
- ad RPI7 в тестовой группу **значимо** выше, чем в контрольной на 3%

Следовательно применять конфигурацию тестовой группы можно.

Команда захочет увидеть этот эффект (увеличение RPI на 3%), когда выкатит это изменение на всех игроков.<br>
**Поэтому важно показывать доверительный интервал для разницы средних между группами.**

In [23]:
from math import sqrt

def confidence_interval_for_diff(data1, data2, a=.05):
    n1 = len(data1)
    n2 = len(data2)
    df = n1 + n2 - 2
    std1 = data1.std()
    std2 = data2.std()
    std_n1n2 = sqrt(((n1 - 1)*std1**2 + (n2 - 1)*std2**2)/df)
    diff_mean = data1.mean() - data2.mean()
    margin_of_error = sp.stats.t.ppf(1 - a/2, df) * std_n1n2 * sqrt(1/n1 + 1/n2)
    
    return margin_of_error

In [24]:
100 * (confidence_interval_for_diff(df_ab[df_ab['ab_group'].str.contains('control')]['rev_ad'], 
                            df_ab[df_ab['ab_group'].str.contains('test')]['rev_ad']) /\
        df_ab[df_ab['ab_group'].str.contains('control')]['rev_ad'].mean() )

2.7785841452643885

*Т.е. по итогу на проде мы можем увидеть эффект по увеличению ad RPI7 как 3.0±2.8%, т.е. от 0.02% до 5.8%.*

### О чем еще стоит знать

1. **Иногда очень важно смотреть на динамику (не по датам, а по мере накопления дней жизни у когорты) определенных метрик.**
<br>Например, если мы задизайнили фичу, которая дает нам буст в первые дни жизни игроков по доходу, однако снижает их ретенш.
Этот эффект от изначального буста у игроков тестовой группы может начать проигровать последовательному доходу 
у игроков из контрольной группы за счет тех, кто не отвалился и продолжил играть. Таким образом, иногда стоит смотреть за метриками более поздних дней, сделав предварительный вывод и сообщив команде о том, что для окончательных выводов необходимо дать пользователям "пожить" еще.
2. **Понятие доверительный интервал для продюсера**
<br>С точки зрения статистики p-value – это не вероятность того, что одна группу лучше другой. Доверительные интервалы не пересекаются – можно очень грубо интерпретровать как, мы уверены в том, что одна группу лучше другой.